# Attrition prediction modeling

## Libraries

In [ ]:
# data wrangling
import pandas as pd
import numpy as np
from scipy.stats.mstats import winsorize

# visuals
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import graphviz 

# preprocessing & 
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as make_imb_pipeline
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import  PolynomialFeatures, StandardScaler, RobustScaler, MinMaxScaler, Normalizer
from sklearn.model_selection import GridSearchCV

# PCA
from sklearn.decomposition import PCA

# ML models
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz

# Deep learning
import joblib
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# feature importance shap values
import eli5
from eli5.sklearn import PermutationImportance
import shap

# metrics
from sklearn.metrics import f1_score, make_scorer, roc_auc_score, roc_curve, auc, accuracy_score
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, ConfusionMatrixDisplay, RocCurveDisplay, plot_confusion_matrix

# system
import warnings
warnings.filterwarnings('ignore')

# settings
pd.set_option('display.max_columns', None)

## Data import

In [ ]:
### data cohort 1
df = pd.read_csv('cohort1.csv')

# rows with NaN values
df[df.isna().any(axis=1)]

# remove all rows with NaN values
df = df.dropna()
#df.head()

### data cohort 1
df2 = pd.read_csv('cohort2.csv')

# rows with NaN values
df2[df2.isna().any(axis=1)]

# remove all rows with NaN values
df2 = df2.dropna()
#df2.head()

## Winsorization

In [ ]:
# store original df for comparison
df_org = df.copy()

### Apply winsorization to train set

In [ ]:
# exlcude variables frequency below winsorization level 
winsorization_col = list(df.columns)
exception_list = ['incoming_transfer_on_investment','outgoing_transfer_on_investment',
                  'profit_on_volume_futures','volume_etf_funds_perc','rec_outgoing_transfer',
                 'volume_futures_perc','is_negative_feedback']
winsorization_col = [x for x in winsorization_col if x not in exception_list]

In [ ]:
# winsorize each column and store min and max values in a new dataframe
winsorize_level = pd.DataFrame(index = ['Min', 'Max'])
lower = 0.01
upper = 0.01

for column in winsorization_col:
    df[column] = winsorize(df[column],(lower,upper))
    winsorize_level[column] = [min(df[column]),max(df[column])]
winsorize_level    

In [ ]:
# plot boxplot for one variable as an example
fig, (ax1, ax2) = plt.subplots(1, 2)
fig.suptitle('Example of variable before and after winsorization (volume_open_closing_ratio)')
ax1.boxplot(df2['volume_open_closing_ratio'])
ax2.boxplot(df['volume_open_closing_ratio'])

ax1.set_xticklabels(['Before'], fontdict=None, minor=False)
ax2.set_xticklabels(['After'], fontdict=None, minor=False)

### Apply winsorization to test set

In [ ]:
for column in winsorization_col:
    winsorize_min = winsorize_level[column].loc['Min']
    winsorize_max = winsorize_level[column].loc['Max']
    df2.loc[df2[column] > winsorize_max, column] = winsorize_max
    df2.loc[df2[column] < winsorize_min, column] = winsorize_min

## EDA

In [ ]:
# describe data
df.describe()

In [ ]:
# calculate attrition
n_cohort1 = df.shape[0]
n_cohort2 = df2.shape[0]

attrition_cohort1 = len(df[(df['is_attrition']==1)])
attrition_cohort2 = len(df2[(df2['is_attrition']==1)])

attrition_rate_cohort1 = attrition_cohort1 / n_cohort1
attrition_rate_cohort2 = attrition_cohort2 / n_cohort2

print('Cohort January 2018 (No. of attrition =',attrition_cohort1,'; n =', n_cohort1,'; attrition rate =',round(attrition_rate_cohort1 * 100,2) ,'%)')
print('Cohort February 2018 (No. of attrition =',attrition_cohort2,'; n =', n_cohort2,'; attrition rate =',round(attrition_rate_cohort2 * 100,2) ,'%)')

In [ ]:
# plot attrition for train and test dataset
list1 = [[0,n_cohort1 - attrition_cohort1], [1,attrition_cohort1]]
list2 = [[0,n_cohort2 - attrition_cohort2], [1,attrition_cohort2]]

dfs = [pd.DataFrame(np.array(lst), 
                    columns=['is_attrition', i]).set_index('is_attrition')
          for i,lst in enumerate([list1,list2])]

dfs_plot = pd.concat(dfs, axis=1)

dfs_plot.plot.bar(color = ['blue','cornflowerblue']).legend(['Train (in-sample)', 'Test (OOP)'])


In [ ]:
# Plot all features to visualize patterns in data
df2.plot(lw=0,
          marker=".",
          subplots=True,
          layout=(-1, 4),
          figsize=(15, 30),
          markersize=1);

In [ ]:
# Histogram for each feature with attrition overlay

# Per class feature histogram
fig, axes = plt.subplots (20, 3, figsize=(20, 70))
fig.suptitle("Feature Distribution" , fontsize=20, y=0.95)
attrition = df.loc[df['is_attrition'] == 1]
no_attrition = df.loc[df['is_attrition'] == 0]

#no_purchase
ax = axes.ravel()
print(ax)

bins = 50

for i in range(54):
    ax[i].hist(no_attrition.iloc[:,i], bins = bins, color = "orange", alpha = 0.5)
    ax[i].hist(attrition.iloc[:,i], bins = bins, color = "blue", alpha = 0.5)
    ax[i].set_title(df.columns[i])

In [ ]:
# plot correlation plot for all feature combinations
# Creates mask to identify numerical features with at least 25 unique features

#cols_continuous = df.select_dtypes(include="number").nunique() >= 10

# Create a new dataframe which only contains the continuous features

#df_continuous = df[cols_continuous[cols_continuous].index]

#sns.pairplot(df_continuous, height=1.5,
#             plot_kws={"s": 2, "alpha": 0.2});

### Remove highly correlated variables

In [ ]:
# Computes feature correlation
df_corr = df.loc[:, ~df.columns.isin(['is_attrition', 'is_female','is_positive_feedback'
                                      ,'is_negative_feedback','is_withdrawal_last_month'
                                      ,'is_withdrawal_last_week','is_deposit_last_month'
                                      ,'is_deposit_last_week']
                                    )].corr(method="pearson",min_periods=1)

mask = np.zeros_like(df_corr)
mask[np.triu_indices_from(mask)] = True

# Create labels for the correlation matrix
labels = np.where(np.abs(df_corr)>0.90, "S",
                  np.where(np.abs(df_corr)>0.5, "M",
                           np.where(np.abs(df_corr)>0.05, "W", "")))

# Plot correlation matrix
plt.figure(figsize=(15, 15))
sns.heatmap(df_corr, mask=mask, square=True,
            center=0, annot=labels, fmt='', linewidths=.5,
            cmap="vlag", cbar_kws={"shrink": 0.8});

In [ ]:
# print correlation matrix
df_corr

In [ ]:
# Above chart shows that num_orders highly correlates with num_BO_SO_orders and num_BC_SC_orders

# drop from train set
df.drop(['num_orders','num_orders_last_month','num_orders_last_week','rec_num_orders'], axis=1, inplace=True)

# drop from test set
df2.drop(['num_orders','num_orders_last_month','num_orders_last_week','rec_num_orders'], axis=1, inplace=True)

#### Plot variables highly correlated with attrition

In [ ]:
plt.figure(figsize=(15, 6))

df_corr = df.loc[:, ~df.columns.isin(['is_female','is_positive_feedback','is_negative_feedback'])].corr(method="pearson",min_periods=1)

attrition_corr = df_corr['is_attrition'].sort_values(ascending=False)[1:].head(55)

attrition_corr.plot.bar()

plt.rcParams['figure.figsize'] = [6, 4]

### Plot histogram and 100% stacked histogram for selected features

In [ ]:
b='blue'  # first default color
o='orange'  # second default color

def plot_hist_norm(x0,x1,bins,title = 'test'):
    
    fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2)

    (n, bin_limits, _) = ax0.hist([x0,x1], bins=10,stacked=False,color=[o,b])
    with np.errstate(divide='ignore',invalid='ignore'):
        n_norm = n / n.sum(axis=0)
    labels=np.round((bin_limits[1:]+bin_limits[:-1])/2,1)    # determine middle of classes
    df_chart=pd.DataFrame({'0':n_norm[0,:],'1':n_norm[1,:]},index=labels)
    df_chart.plot.bar(ax=ax1, stacked=True, figsize=(14, 2),color=[o,b])
    ax0.set_title(title + ' histogram')
    ax1.set_title(title + ' histogram normalized')
    ax1.legend(['1', '0'])

In [ ]:
column_name = attrition_corr.index[:6]
for col in column_name:
    attr_data=df.loc[df.is_attrition==1,col]
    no_attr_data=df.loc[df.is_attrition==0,col]
    plot_hist_norm(attr_data,no_attr_data,10,col)

### Add dummy variables

In [ ]:
# train
df.loc[df['withdrawal_eur_last_month'] > 0, 'is_withdrawal_last_month'] = 1
df.loc[df['withdrawal_eur_last_month'] == 0, 'is_withdrawal_last_month'] = 0

df.loc[df['withdrawal_eur_last_week'] > 0, 'is_withdrawal_last_week'] = 1
df.loc[df['withdrawal_eur_last_week'] == 0, 'is_withdrawal_last_week'] = 0

df.loc[df['deposit_eur_last_month'] > 0, 'is_deposit_last_month'] = 1
df.loc[df['deposit_eur_last_month'] == 0, 'is_deposit_last_month'] = 0

df.loc[df['deposit_eur_last_week'] > 0, 'is_deposit_last_week'] = 1
df.loc[df['deposit_eur_last_week'] == 0, 'is_deposit_last_week'] = 0

# test
df2.loc[df['withdrawal_eur_last_month'] > 0, 'is_withdrawal_last_month'] = 1
df2.loc[df['withdrawal_eur_last_month'] == 0, 'is_withdrawal_last_month'] = 0

df2.loc[df['withdrawal_eur_last_week'] > 0, 'is_withdrawal_last_week'] = 1
df2.loc[df['withdrawal_eur_last_week'] == 0, 'is_withdrawal_last_week'] = 0

df2.loc[df['deposit_eur_last_month'] > 0, 'is_deposit_last_month'] = 1
df2.loc[df['deposit_eur_last_month'] == 0, 'is_deposit_last_month'] = 0

df2.loc[df['deposit_eur_last_week'] > 0, 'is_deposit_last_week'] = 1
df2.loc[df['deposit_eur_last_week'] == 0, 'is_deposit_last_week'] = 0

In [ ]:
df.describe().T.to_csv("descriptive_stats.csv")

### Plot for upsampling

In [ ]:
# create plot for upsampling
list1 = [[0,n_cohort1 - attrition_cohort1], [1,attrition_cohort1]]
list2 = [[0,n_cohort1 - attrition_cohort1], [1,n_cohort1 - attrition_cohort1]]

dfs = [pd.DataFrame(np.array(lst), 
                    columns=['is_attrition', i]).set_index('is_attrition')
          for i,lst in enumerate([list1,list2])]

dfs_plot = pd.concat(dfs, axis=1)

dfs_plot.plot.bar(color = ['orangered','mediumorchid']).legend(['Original', 'Upsampled'], loc = 'upper center')

## Train-test data split

In [ ]:
# define train and test dataset

X_train = df.loc[:, ~df.columns.isin(['is_attrition'])]
y_train = df['is_attrition']
X_test = df2.loc[:, ~df2.columns.isin(['is_attrition'])]
y_test = df2['is_attrition']

# split train dataset to train and valiadtion dataset. 
X_trainD, X_valid, y_trainD, y_valid = train_test_split(X_train, y_train, test_size=0.33, random_state=3, stratify=y_train)



## PCA

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

pca = PCA(n_components=10)
X_train_pca = pca.fit_transform(X_train_scaled)

print(X_train.shape)
print(X_train_pca.shape)

In [ ]:
# plot first 2 PCAs
df_plot_pca = pd.DataFrame({'1_pca':X_train_pca[:,0], '2_pca':X_train_pca[:,1], '3_pca':X_train_pca[:,2], 'y':y_train})

plt.figure(figsize=(8, 8))
no_purchase = plt.scatter(df_plot_pca.loc[df_plot_pca['y'] == 0, ['1_pca']], df_plot_pca.loc[df_plot_pca['y'] == 0, ['2_pca']], marker='o', color="orange", alpha=0.5, label='no_purchase')
purchase = plt.scatter(df_plot_pca.loc[df_plot_pca['y'] == 1, ['1_pca']], df_plot_pca.loc[df_plot_pca['y'] == 1, ['2_pca']], marker='^', color="blue", alpha=0.2, label='purchase')

plt.legend(loc='upper left')
plt.xlabel("First principal component")
plt.ylabel("Second principal component")



In [ ]:
#from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')

ax.scatter(df_plot_pca.loc[df_plot_pca['y'] == 1, ['1_pca']], df_plot_pca.loc[df_plot_pca['y'] == 1, ['2_pca']], df_plot_pca.loc[df_plot_pca['y'] == 1, ['3_pca']], marker='^', color="blue", alpha=0.2, label='purchase')
ax.scatter(df_plot_pca.loc[df_plot_pca['y'] == 0, ['1_pca']], df_plot_pca.loc[df_plot_pca['y'] == 0, ['2_pca']], df_plot_pca.loc[df_plot_pca['y'] == 0, ['3_pca']], marker='o', color="orange", alpha=0.5, label='no_purchase')

plt.legend(loc='upper left')
ax.set_xlabel("First principal component")
ax.set_ylabel("Second principal component")
ax.set_zlabel('Third principal component')

# rotate
ax.view_init(-140, 100)

In [ ]:
plt.matshow(pca.components_,cmap='viridis')
#plt.yticks([0,1,2],['First component','Second component','Third component'])
plt.yticks(range(10),range(1,11))
plt.colorbar()
plt.xticks(range(len(X.columns)),X.columns, rotation=60, ha='left')
plt.xlabel('Feature')
plt.ylabel('Principal components')

In [ ]:
# Scree plot

plt.figure(figsize=(18 , 10))

plt.plot(pca.explained_variance_ratio_,'o')
plt.plot(pca.explained_variance_ratio_)
plt.xticks(range(0,10,1),range(1,11))
plt.xlabel('Principle Components')
plt.ylabel('Eigen Values')
plt.show()

## ML Models

### Helper Functions

In [ ]:
# function for displaying confusion matrix, ROC and classification report

def plot_confusion_matrix(best_estimator ,name = 'Name not defined'):
    
    classifier = best_estimator

    # Confusion matrix
    title_options = [
        ("Confusion matrix, without normalization", None),
        ("Normalized confusion matrix", "true"),
    ]
    for title, normalize in title_options:
        display = ConfusionMatrixDisplay.from_estimator(
            classifier,
            X_test,
            y_test,
            cmap=plt.cm.Blues,
            normalize=normalize,
        )
        display.ax_.set_title(title)

        #print(title)
        #print(display.confusion_matrix)
      
    # ROC curve
    RocCurveDisplay.from_estimator(best_estimator, X_test, y_test, name = name)
    
    # Classification report
    y_pred = best_estimator.predict(X_test)
    print('')
    print('Classification Report')    
    print(classification_report(y_test, y_pred))
    
    plt.show()

In [ ]:
# function for displaying confusion matrix, ROC and classification report

def plot_roc_curve(best_estimator ,name = 'Name not defined'):
    
    plt.rcParams['figure.figsize'] = [6, 4]
    classifier = best_estimator
      
    # ROC curve
    RocCurveDisplay.from_estimator(best_estimator, X_test, y_test, name = name)
    
    
    plt.show()

### Random Forest (RF)

#### Model

In [ ]:
# create pipeline
pipe = make_imb_pipeline(RandomOverSampler(sampling_strategy='not majority', random_state=2), Normalizer(), RandomForestClassifier(warm_start=True, random_state=2))

# cross validations
cv = 3


param_grid = {"randomforestclassifier__n_estimators": range(100)
            ,'randomforestclassifier__max_depth' : [2,3,4,5,6,7,8,10]
            ,'randomforestclassifier__criterion' :['entropy']
            
             }

grid_rf = GridSearchCV(pipe, param_grid, return_train_score=True, cv=cv
                       , scoring= 'f1_macro'
                       ,n_jobs=-1)
grid_rf.fit(X_train, y_train)
#grid_rf.fit(X, y)



print("best mean cross-validation score: {:.3f}".format(grid_rf.best_score_))
print("best parameters: {}".format(grid_rf.best_params_))

#### Validation Curve

In [ ]:
## Plot the validation curve 
rf_p_results = pd.DataFrame(grid_rf.cv_results_)

rf_p_results = rf_p_results[rf_p_results['param_randomforestclassifier__criterion']=='entropy']
rf_p_results = rf_p_results[rf_p_results['param_randomforestclassifier__n_estimators']==39]

plt.rcParams['figure.figsize'] = [7, 5]
ax = plt.gca()
ax.plot(rf_p_results["param_randomforestclassifier__max_depth"], rf_p_results["mean_train_score"], label='Training Score')
ax.plot(rf_p_results["param_randomforestclassifier__max_depth"], rf_p_results["mean_test_score"], label='Cross-Validation Score')
plt.xlabel("max depth")
plt.ylabel("Score")
plt.title("Validation Random Forest")
#plt.axis("tight")
plt.legend()
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = [8, 15]

plt.matshow(grid_rf.cv_results_['mean_test_score'].reshape(8,-1),
            vmin=0, cmap="viridis")

plt.xlabel("n_estimators")
plt.ylabel("max_depth")
plt.yticks(range(len(param_grid['randomforestclassifier__max_depth'])), param_grid['randomforestclassifier__max_depth'])
plt.colorbar();

plt.rcParams['figure.figsize'] = [6, 4]

#### Confusion matrix & ROC

In [ ]:
plot_confusion_matrix(grid_rf.best_estimator_, 'Random Forest')

#### Feature importance

In [ ]:
plt.rcParams['figure.figsize'] = [10, 14]
feature_importances = grid_rf.best_estimator_.named_steps["randomforestclassifier"].feature_importances_

feat_importances = pd.Series(feature_importances, index=X_train.columns)
#feat_importances = pd.Series(feature_importances, index=X.columns)
feat_importances.nlargest(25).plot(kind='barh')
plt.title("Features importances (Random Forest model)")
plt.show()
plt.rcParams['figure.figsize'] = [6, 4]

feat_importances_rf = feat_importances

In [ ]:
best_estimator = grid_rf.best_estimator_
y_pred = best_estimator.predict(X_test)

#pd.set_option('precision', 2)
pd.options.display.float_format = "{:,.3f}".format
# merge results

#y_pred.shape
overview = X_test.copy(deep=True)

overview['pred_attrition'] = y_pred.tolist()

overview = overview.groupby(['pred_attrition']).mean().T

In [ ]:
overview['feat_importance'] = feat_importances.tolist()
overview.nlargest(35, columns = 'feat_importance')

### Logistic Regression

#### Model

In [ ]:
# create pipeline
pipe = make_imb_pipeline(RandomOverSampler(sampling_strategy='not majority', random_state=2), Normalizer(), LogisticRegression(multi_class = 'auto', solver = 'lbfgs',random_state = 2, max_iter=10000))

param_grid = {"logisticregression__C": np.logspace(-5,7)}
grid_log = GridSearchCV(pipe, param_grid, return_train_score=True, cv=3, scoring= 'f1_macro', n_jobs=-1)
grid_log.fit(X_train, y_train);

print("best mean cross-validation score: {:.3f}".format(grid_log.best_score_))
print("best parameters: {}".format(grid_log.best_params_))

#### Validation Curve

In [ ]:
## Plot the validation curve
log_results = pd.DataFrame(grid_log.cv_results_)

#log_results = log_results[log_results['param_polynomialfeatures__degree']==1]

plt.rcParams['figure.figsize'] = [7, 5]
ax = plt.gca()
ax.plot(log_results["param_logisticregression__C"], log_results["mean_train_score"], label='Training Score')
ax.plot(log_results["param_logisticregression__C"], log_results["mean_test_score"], label='Cross-Validation Score')
ax.set_xscale("log")
plt.xlabel("C")
plt.ylabel("Score")
plt.title("Validation Curve Logistic Regression")
plt.axis("tight")
plt.legend()
plt.show()

#### Confusion matrix & ROC

In [ ]:
plot_confusion_matrix(grid_log.best_estimator_, 'Logistic Regression')

#### Feature importance

In [ ]:
plt.rcParams['figure.figsize'] = [12, 20]

# get the best performing model fit on the whole training set
best_model_log = grid_log.best_estimator_
#best_model_log.steps[3]
feat_importances = best_model_log.named_steps['logisticregression'].coef_[0]
#importance = best_model_log.coef_[0]

#importance[0]

#feat_importances = pd.Series(importance)

#feat_importances = pd.Series(importance[1:], index=X_train.columns)
feat_importances = pd.Series(feat_importances, index=X_train.columns)

feat_importances.nlargest(55).plot(kind='barh',title = 'Feature Importance')

plt.rcParams['figure.figsize'] = [6, 4]

feat_importances_logr = feat_importances

In [ ]:
best_estimator = grid_log.best_estimator_
y_pred = best_estimator.predict(X_test)

#pd.set_option('precision', 2)
pd.options.display.float_format = "{:,.3f}".format
# merge results

#y_pred.shape
overview = X_test.copy(deep=True)

overview['pred_attrition'] = y_pred.tolist()

overview = overview.groupby(['pred_attrition']).mean().T

overview['feat_importance'] = feat_importances.tolist()
overview.nlargest(55, columns = 'feat_importance')

### Linear SVC

#### Model

In [ ]:
# create pipeline
pipe = make_imb_pipeline(RandomOverSampler(sampling_strategy='not majority', random_state=2), Normalizer(), LinearSVC(dual = False, random_state=2, max_iter=1000))

param_grid = {"linearsvc__C": np.logspace(-2,10,8)}
grid_svc = GridSearchCV(pipe, param_grid, return_train_score=True, cv=3, scoring= 'f1_macro',  n_jobs=-1)
grid_svc.fit(X_train, y_train)

print("best mean cross-validation score: {:.3f}".format(grid_svc.best_score_))
print("best parameters: {}".format(grid_svc.best_params_))

#### Validation Curve

In [ ]:
## Plot the validation curve
svc_results = pd.DataFrame(grid_svc.cv_results_)

#svc_results = svc_results[svc_results['param_polynomialfeatures__degree']==1]

plt.rcParams['figure.figsize'] = [7, 5]
ax = plt.gca()
ax.plot(svc_results["param_linearsvc__C"], svc_results["mean_train_score"], label='Training Score')
ax.plot(svc_results["param_linearsvc__C"], svc_results["mean_test_score"], label='Cross-Validation Score')
ax.set_xscale("log")
plt.xlabel("C")
plt.ylabel("Score")
plt.title("Validation Curve Linear SVC")
plt.axis("tight")
plt.legend()
plt.show()

#### Confusion matrix & ROC

In [ ]:
plot_confusion_matrix(grid_svc.best_estimator_, 'Linear SVC')

#### Feature Importance

In [ ]:
best_estimator = grid_svc.best_estimator_
y_pred = best_estimator.predict(X_test)

plt.rcParams['figure.figsize'] = [12, 20]

#best_model_log.steps[3]
feat_importances = best_estimator.named_steps['linearsvc'].coef_[0]

feat_importances = pd.Series(feat_importances, index=X_train.columns)

feat_importances.nlargest(55).plot(kind='barh',title = 'Feature Importance')

plt.rcParams['figure.figsize'] = [6, 4]

feat_importances_svc = feat_importances

In [ ]:
#pd.set_option('precision', 2)
pd.options.display.float_format = "{:,.3f}".format
# merge results

#y_pred.shape
overview = X_test.copy(deep=True)

overview['pred_attrition'] = y_pred.tolist()

overview = overview.groupby(['pred_attrition']).mean().T

overview['feat_importance'] = feat_importances.tolist()
overview.nlargest(55, columns = 'feat_importance')

### KNN

#### Model

In [ ]:
# create pipeline
pipe = make_imb_pipeline(RandomOverSampler(sampling_strategy='not majority', random_state=2), Normalizer(), KNeighborsClassifier())

param_grid = {'kneighborsclassifier__n_neighbors': range(1, 20)}
grid_knn = GridSearchCV(pipe, param_grid, return_train_score=True, cv=3, scoring= 'f1_macro',  n_jobs=-1)
grid_knn.fit(X_train, y_train)

print("best mean cross-validation score: {:.3f}".format(grid_knn.best_score_))
print("best parameters: {}".format(grid_knn.best_params_))

#### Validation Curve

In [ ]:
## Plot the validation curve
knn_results = pd.DataFrame(grid_knn.cv_results_)

ax = plt.gca()
ax.plot(knn_results["param_kneighborsclassifier__n_neighbors"], knn_results["mean_train_score"], label='Training Score')
ax.plot(knn_results["param_kneighborsclassifier__n_neighbors"], knn_results["mean_test_score"], label='Cross-Validation Score')
#ax.set_xscale("log")
plt.xlabel("n_neighbors")
plt.ylabel("Score")
plt.title("Validation Curve KNN")
plt.axis("tight")
plt.legend()
plt.show()

#### Confusion matrix & ROC

In [ ]:
plot_confusion_matrix(grid_knn.best_estimator_, 'KNN')

### Bayes Classification

#### Model

In [ ]:
# create pipeline
from sklearn.naive_bayes import GaussianNB

pipe = make_imb_pipeline(RandomOverSampler(sampling_strategy='not majority', random_state=2), Normalizer(), GaussianNB())

param_grid = {}
grid_bayes = GridSearchCV(pipe, param_grid, return_train_score=True, cv=3, scoring= 'f1_macro',  n_jobs=-1)
grid_bayes.fit(X_train, y_train)

print("best mean cross-validation score: {:.3f}".format(grid_bayes.best_score_))
print("best parameters: {}".format(grid_bayes.best_params_))

#### Confusion matrix & ROC

In [ ]:
plot_confusion_matrix(grid_bayes.best_estimator_, 'Bayes Classification')

### Decision Tree CART

#### Model

In [ ]:

criterion = 'gini'

pipe = make_imb_pipeline(RandomOverSampler(sampling_strategy='not majority', random_state=2), Normalizer(), DecisionTreeClassifier(criterion = criterion, random_state = 2))

param_grid = {'decisiontreeclassifier__max_depth': range(1,30,5), 'decisiontreeclassifier__min_samples_leaf': range(1,30,5)}
grid_cart = GridSearchCV(pipe, param_grid, return_train_score=True, cv=3, scoring= 'f1_macro',  n_jobs=-1)
grid_cart.fit(X_train, y_train)

print("best mean cross-validation score: {:.3f}".format(grid_cart.best_score_))
print("best parameters: {}".format(grid_cart.best_params_))

#### Validation Curve

In [ ]:
result_df = pd.DataFrame.from_dict(grid_cart.cv_results_, orient='columns')
print(result_df.columns)

In [ ]:
sns.relplot(data=result_df,
    kind='line',
    x='param_decisiontreeclassifier__max_depth',
    y='mean_test_score',
#    hue='param_scaler',
    col='param_decisiontreeclassifier__min_samples_leaf')
plt.show()

#### Confusion matrix & ROC

In [ ]:
plot_confusion_matrix(grid_cart.best_estimator_, 'Decision Tree CART')

#### Decision tree plot

In [ ]:
dot_data = export_graphviz(grid_cart.best_estimator_[2], out_file=None, 
            filled=True, rounded=True, feature_names=X_train.columns, class_names=['0','1'])
graph = graphviz.Source(dot_data)   
graph

### Decision Tree C5.0

#### Model

In [ ]:
criterion = 'entropy'

pipe = make_imb_pipeline(RandomOverSampler(sampling_strategy='not majority', random_state=2), Normalizer(), DecisionTreeClassifier(criterion = criterion, random_state = 2))

param_grid = {'decisiontreeclassifier__max_depth': range(1,30), 'decisiontreeclassifier__min_samples_leaf': range(1,30)}
grid_c50 = GridSearchCV(pipe, param_grid, return_train_score=True, cv=3, scoring= 'f1_macro',  n_jobs=-1)
grid_c50.fit(X_train, y_train)

print("best mean cross-validation score: {:.3f}".format(grid_c50.best_score_))
print("best parameters: {}".format(grid_c50.best_params_))

#### Confusion matrix & ROC

In [ ]:
plot_confusion_matrix(grid_c50.best_estimator_, 'Decision Tree C5.0')

#### Decision tree plot

In [ ]:
dot_data = export_graphviz(grid_c50.best_estimator_[2], out_file=None, 
            filled=True, rounded=True, feature_names=X_train.columns, class_names=['0','1'])
graph = graphviz.Source(dot_data)   
graph

### GradientBoost

In [ ]:
# create pipe
pipe = make_imb_pipeline(RandomOverSampler(sampling_strategy='not majority', random_state=2), Normalizer(), GradientBoostingClassifier(warm_start=True, random_state=2))

# cross validations
cv = 3


param_grid = {"gradientboostingclassifier__n_estimators": [50,60,70,80]
            ,'gradientboostingclassifier__max_depth' : [6,7,8,9]
            ,'gradientboostingclassifier__learning_rate' : [0.1]
            
             }

grid_gb = GridSearchCV(pipe, param_grid, return_train_score=True, cv=cv
                       , scoring= 'f1_macro'
                       ,n_jobs=-1)
grid_gb.fit(X_train, y_train)


print("best mean cross-validation score: {:.3f}".format(grid_gb.best_score_))
print("best parameters: {}".format(grid_gb.best_params_))

In [ ]:
plot_confusion_matrix(grid_gb.best_estimator_, 'Gradient Boosting')

#### Feature importance

In [ ]:
best_estimator = grid_gb.best_estimator_
y_pred = best_estimator.predict(X_test)

plt.rcParams['figure.figsize'] = [12, 20]

#best_model_log.steps[3]
feat_importances = best_estimator.named_steps['gradientboostingclassifier'].feature_importances_

feat_importances = pd.Series(feat_importances, index=X_train.columns)

feat_importances.nlargest(55).plot(kind='barh',title = 'Feature Importance')

plt.rcParams['figure.figsize'] = [6, 4]

feat_importances_gb = feat_importances

In [ ]:
#pd.set_option('precision', 2)
pd.options.display.float_format = "{:,.3f}".format
# merge results

#y_pred.shape
overview = X_test.copy(deep=True)

overview['pred_attrition'] = y_pred.tolist()

overview = overview.groupby(['pred_attrition']).mean().T

overview['feat_importance'] = feat_importances.tolist()
overview.nlargest(55, columns = 'feat_importance')

#### Lift Curve

In [ ]:
def plot_lift_curve(y_val, y_pred, step=0.01):
    
    #Define an auxiliar dataframe to plot the curve
    aux_lift = pd.DataFrame()
    #Create a real and predicted column for our new DataFrame and assign values
    aux_lift['real'] = y_val
    aux_lift['predicted'] = y_pred
    #Order the values for the predicted probability column:
    aux_lift.sort_values('predicted',ascending=False,inplace=True)
    
    #Create the values that will go into the X axis of our plot
    x_val = np.arange(step,1+step,step)
    #Calculate the ratio of ones in our data
    ratio_ones = aux_lift['real'].sum() / len(aux_lift)
    #Create an empty vector with the values that will go on the Y axis our our plot
    y_v = []
    
    #Calculate for each x value its correspondent y value
    for x in x_val:
        num_data = int(np.ceil(x*len(aux_lift))) #The ceil function returns the closest integer bigger than our number 
        data_here = aux_lift.iloc[:num_data,:]   # ie. np.ceil(1.4) = 2
        ratio_ones_here = data_here['real'].sum()/len(data_here)
        y_v.append(ratio_ones_here / ratio_ones)
           
   #Plot the figure
    fig, axis = plt.subplots()
    fig.figsize = (60,60)
    axis.plot(x_val, y_v, 'g-', linewidth = 3, markersize = 5)
    axis.plot(x_val, np.ones(len(x_val)), 'k-')
    axis.set_xlabel('Proportion of sample')
    axis.set_ylabel('Lift')
    plt.title('Lift Curve')
    plt.show()
    plt.rcParams['figure.figsize'] = [6, 4]
    
    return y_v

In [ ]:
plot_lift_curve(y_test, y_pred, 0.1)

### Deep Learning

#### Work with data imbalance and scaling

In [ ]:
# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy='not majority', random_state=2)
# fit and apply the transform
X_trainD, y_trainD = oversample.fit_resample(X_trainD, y_trainD)

In [ ]:
# scaling
scaler = Normalizer()
X_trainD_norm = scaler.fit_transform(X_trainD)
X_valid_norm = scaler.transform(X_valid)
X_test_norm = scaler.transform(X_test)

In [ ]:
X_trainD_norm.shape

#### ANN find best model

In [ ]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[53]): 
    model = keras.models.Sequential()
    options = {"input_shape": input_shape}
    for layer in range(n_hidden):
            model.add(keras.layers.Dense(n_neurons, activation="relu", **options))
            options = {} 
    model.add(keras.layers.Dense(1, **options)) 
    #optimizer = keras.optimizers.SGD(learning_rate)
    optimizer='adam'
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True)
    model.compile(loss=loss, optimizer=optimizer,metrics=['accuracy'])
    #model.compile(loss="mse", optimizer=optimizer) 
    return model

In [ ]:
keras_reg = keras.wrappers.scikit_learn.KerasClassifier(build_model)

In [ ]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
      #  "n_hidden": range(1,10,2),
      #  "n_neurons": np.arange(1, 40),
        "n_hidden": [9],
        "n_neurons": [39],
        "learning_rate": reciprocal(3e-4, 3e-2),
}
rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3)
rnd_search_cv.fit(X_trainD_norm, y_trainD, epochs=100,
                  validation_data=(X_valid_norm, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

In [ ]:
rnd_search_cv.best_params_

In [ ]:
rnd_search_cv.best_score_

In [ ]:
model_ann = rnd_search_cv.best_estimator_.model

In [ ]:
y_proba = model_ann.predict(X_test_norm)
y_score = np.array(y_proba)#[:,1]
fpr, tpr, thresholds = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)

display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc,
                                   estimator_name='ANN')
display.plot()

plt.show()

In [ ]:
y_pred = model_ann.predict(X_test_norm)
auc = round(metrics.roc_auc_score(y_test, y_pred), 4)
y_pred_binary = (model_ann.predict(X_test_norm) > 0.5).astype("int32")
print('AUC:',auc)
print('Accuracy:',accuracy_score(y_test, y_pred_binary))
print(classification_report(y_test, y_pred_binary))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred_binary)

#### Feature Importance

In [ ]:
shap.initjs()

#explainer = shap.DeepExplainer(model_ann,X_trainD_norm)
#explainer = shap.PermutationExplainer(model_ann,X_trainD_norm[:50,:])
explainer = shap.KernelExplainer(model_ann,X_trainD_norm[:50,:])



In [ ]:
X_trainD_norm[50:500,:]

In [ ]:
shap_values = explainer.shap_values(X_trainD_norm[50:70,:])
#shap_values = explainer

In [ ]:
shap.force_plot(explainer.expected_value, shap_values[0], X_trainD_norm[50:70,:])


In [ ]:
shap.summary_plot(shap_values, X_train, plot_type="bar")

### Summary Models

#### ROC curves

In [ ]:
plt.rcParams['figure.figsize'] = [12, 10]


estimators = {'Decision tree C5.0' : grid_c50.best_estimator_,
             'Decision Tree CART' : grid_cart.best_estimator_,
              'Bayes Classification' : grid_bayes.best_estimator_,
              'KNN' : grid_knn.best_estimator_,
              'Linear SVC' : grid_svc.best_estimator_,
              'Logistic Regression' : grid_log.best_estimator_,
              'Random Forest' : grid_rf.best_estimator_,
              'ANN' : model_ann,
              'Gradient Boost' : grid_gb.best_estimator_,
             }

for key in estimators:

    best_estimator = estimators[key]
    
    if key in ['Linear SVC']:
        y_pred = best_estimator.decision_function(X_test)
    elif key in ['ANN']:
        y_pred = best_estimator.predict(X_test_norm)
    else:
        y_pred = best_estimator.predict_proba(X_test)[:,1]
    fpr, tpr, _ = metrics.roc_curve(y_test, y_pred)
    auc = round(metrics.roc_auc_score(y_test, y_pred), 4)
    plt.plot(fpr,tpr,label=key+', AUC='+str(auc))

plt.legend(loc=0)
plt.show()

plt.rcParams['figure.figsize'] = [6, 4]

#### Overview of results

In [ ]:
models_overview = pd.DataFrame(columns=['Accuracy score train','Accuracy score test','F1 score train','F1 score test','AuC_train','AuC_test'])

estimators = {'Decision tree C5.0' : grid_c50.best_estimator_,
             'Decision Tree CART' : grid_cart.best_estimator_,
              'Bayes Classification' : grid_bayes.best_estimator_,
              'KNN' : grid_knn.best_estimator_,
              'Linear SVC' : grid_svc.best_estimator_,
              'Logistic Regression' : grid_log.best_estimator_,
              'Random Forest' : grid_rf.best_estimator_,
              'ANN' : model_ann,
              'Gradient Boost' : grid_gb.best_estimator_,
             }

# AuC test set
for key in estimators:

    best_estimator = estimators[key]
    
    if key in ['Linear SVC']:
        y_pred = best_estimator.decision_function(X_test)
    elif key in ['ANN']:
        y_pred = best_estimator.predict(X_test_norm)
    else:
        y_pred = best_estimator.predict_proba(X_test)[:,1]
    fpr, tpr, _ = metrics.roc_curve(y_test, y_pred)
    auc = round(metrics.roc_auc_score(y_test, y_pred), 4)
    #f1score = f1_score(y_test, y_pred)
    
    
    #models_overview.loc[key] = pd.Series({'AuC_test':auc})
    models_overview.loc[key,['AuC_test']] = auc
    #models_overview.loc[key,['F1 score test']] = f1score

    # AuC train set
for key in estimators:

    best_estimator = estimators[key]
    
    if key in ['Linear SVC']:
        y_pred = best_estimator.decision_function(X_train)
    elif key in ['ANN']:
        y_pred = best_estimator.predict(X_trainD_norm)
    else:
        y_pred = best_estimator.predict_proba(X_train)[:,1]
    
    if key in ['ANN']:
        auc = round(metrics.roc_auc_score(y_trainD, y_pred), 4)
        #f1score = f1_score(y_trainD, y_pred)
    else:
        auc = round(metrics.roc_auc_score(y_train, y_pred), 4)
        #f1score = f1_score(y_train, y_pred)
    
    models_overview.loc[key,['AuC_train']] = auc
    #models_overview.loc[key,['F1 score train']] = f1score
    
# F1 score 
for key in estimators:

    # test data f1
    best_estimator = estimators[key]
    if key in ['ANN']:
        y_pred = (best_estimator.predict(X_test_norm) > 0.5).astype("int32")
    else:
        y_pred = best_estimator.predict(X_test)

    models_overview.loc[key,['F1 score test']] = f1_score(y_test, y_pred)
    
    # train data f1
    best_estimator = estimators[key]
    if key in ['ANN']:
        y_pred = (best_estimator.predict(X_trainD_norm) > 0.5).astype("int32")
        models_overview.loc[key,['F1 score train']] = f1_score(y_trainD, y_pred)

    else:
        y_pred = best_estimator.predict(X_train)
        models_overview.loc[key,['F1 score train']] = f1_score(y_train, y_pred)    
    
# Accuracy score 
for key in estimators:

    # test data f1
    best_estimator = estimators[key]
    if key in ['ANN']:
        y_pred = (best_estimator.predict(X_test_norm) > 0.5).astype("int32")
    else:
        y_pred = best_estimator.predict(X_test)

    models_overview.loc[key,['Accuracy score test']] = accuracy_score(y_test, y_pred)
    
    # train data f1
    best_estimator = estimators[key]
    if key in ['ANN']:
        y_pred = (best_estimator.predict(X_trainD_norm) > 0.5).astype("int32")
        models_overview.loc[key,['Accuracy score train']] = accuracy_score(y_trainD, y_pred)

    else:
        y_pred = best_estimator.predict(X_train)
        models_overview.loc[key,['Accuracy score train']] = accuracy_score(y_train, y_pred)  
    

In [ ]:
#models_overview.index.name = 'Model'
models_overview[['Accuracy score test','F1 score test','AuC_test']].sort_values(by='AuC_test', ascending=False)

## prediction distribution for best model

In [ ]:
y_pred = grid_gb.best_estimator_.predict_proba(X_test)[:,1]

In [ ]:
y_pred_df = pd.DataFrame(y_pred)

y_pred_df.hist(bins = 20)

In [ ]:
plt.rcParams['figure.figsize'] = [6,6]
plt.xlim(0, 1)
sns.set_style('white')
sns.distplot(y_pred_df, kde=False, color="b", bins = 15).set(title='Probability Distribution of Attrition Risk')
plt.rcParams['figure.figsize'] = [6, 4]

## feature importance chart

In [ ]:
feat_importances_gb
feat_importances_svc
feat_importances_rf
feat_importances_logr

In [ ]:
plot_data = pd.concat([feat_importances_gb,feat_importances_rf,feat_importances_svc,feat_importances_logr], axis=1)
plot_data = plot_data.sort_values(plot_data.columns[0], ascending = False).head(10)
plot_data = plot_data.set_axis(['Gradient Boost', 'Random Forest', 'Linear SVC', 'Logistic Regression'], axis=1, inplace=False)
plot_data_rf = plot_data

In [ ]:
plot_data = plot_data.set_axis(['Gradient Boost', 'Random Forest', 'Linear SVC', 'Logistic Regression'], axis=1, inplace=False)
plot_data

In [ ]:
plt.rcParams['figure.figsize'] = [15, 6]

f, (ax1, ax2, ax3, ax4) = plt.subplots(1,4)
sns.barplot(x=plot_data.columns[0], y=plot_data.index, data=plot_data, ax = ax1,
            label="Total", color="b")
sns.barplot(x=plot_data.columns[1], y=plot_data.index, data=plot_data, ax = ax2,
            label="Total", color="b")
sns.barplot(x=plot_data.columns[2], y=plot_data.index, data=plot_data, ax = ax3,
            label="Total", color="b")
sns.barplot(x=plot_data.columns[3], y=plot_data.index, data=plot_data, ax = ax4,
            label="Total", color="b")

ax2.set(yticklabels=[])
ax3.set(yticklabels=[])
ax4.set(yticklabels=[])

plt.rcParams['figure.figsize'] = [6, 4]

In [ ]:
plot_data = pd.concat([feat_importances_gb,feat_importances_rf,feat_importances_svc,feat_importances_logr], axis=1)
plot_data = plot_data.sort_values(plot_data.columns[2], ascending = False).head(10)
plot_data = plot_data.set_axis(['Gradient Boost', 'Random Forest', 'Linear SVC', 'Logistic Regression'], axis=1, inplace=False)
plot_data_svc_pos = plot_data

In [ ]:
plt.rcParams['figure.figsize'] = [10, 4]

f, (ax3, ax4) = plt.subplots(1,2)
sns.barplot(x=plot_data.columns[2], y=plot_data.index, data=plot_data, ax = ax3,
            label="Total", color="b")
sns.barplot(x=plot_data.columns[3], y=plot_data.index, data=plot_data, ax = ax4,
            label="Total", color="b")


ax4.set(yticklabels=[])

plt.rcParams['figure.figsize'] = [6, 4]

In [ ]:
plot_data = pd.concat([feat_importances_gb,feat_importances_rf,feat_importances_svc,feat_importances_logr], axis=1)
plot_data = plot_data.sort_values(plot_data.columns[2], ascending = True).head(10)
plot_data = plot_data.set_axis(['Gradient Boost', 'Random Forest', 'Linear SVC', 'Logistic Regression'], axis=1, inplace=False)
plot_data_svc_neg = plot_data

In [ ]:
plt.rcParams['figure.figsize'] = [10, 4]

f, (ax3, ax4) = plt.subplots(1,2)
sns.barplot(x=plot_data.columns[2], y=plot_data.index, data=plot_data, ax = ax3,
            label="Total", color="r")
sns.barplot(x=plot_data.columns[3], y=plot_data.index, data=plot_data, ax = ax4,
            label="Total", color="r")


ax4.set(yticklabels=[])

plt.rcParams['figure.figsize'] = [6, 4]

In [ ]:
plot_data = pd.concat([feat_importances_gb,feat_importances_rf,feat_importances_svc,feat_importances_logr], axis=1)
plot_data = plot_data.sort_values(plot_data.columns[1], ascending = False).head(20)
plot_data = plot_data.set_axis(['Gradient Boost', 'Random Forest', 'Linear SVC', 'Logistic Regression'], axis=1, inplace=False)

In [ ]:
plt.rcParams['figure.figsize'] = [10, 6]

f, (ax1, ax2) = plt.subplots(1,2)
sns.barplot(x=plot_data.columns[1], y=plot_data.index, data=plot_data, ax = ax1,
            label="Total", color="b")
sns.barplot(x=plot_data.columns[0], y=plot_data.index, data=plot_data, ax = ax2,
            label="Total", color="b")


ax2.set(yticklabels=[])


plt.rcParams['figure.figsize'] = [6, 4]

In [ ]:
plt.rcParams['figure.figsize'] = [13, 10]

f, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3,2)
sns.barplot(x=plot_data.columns[0], y=plot_data.index, data=plot_data, ax = ax1,
            label="Total", color="b")
sns.barplot(x=plot_data.columns[1], y=plot_data.index, data=plot_data, ax = ax2,
            label="Total", color="b")
sns.barplot(x=plot_data_svc_pos.columns[2], y=plot_data_svc_pos.index, data=plot_data_svc_pos, ax = ax3,
            label="Total", color="b")
sns.barplot(x=plot_data_svc_pos.columns[3], y=plot_data_svc_pos.index, data=plot_data_svc_pos, ax = ax4,
            label="Total", color="b")
sns.barplot(x=plot_data_svc_neg.columns[2], y=plot_data_svc_neg.index, data=plot_data_svc_neg, ax = ax5,
            label="Total", color="r")
sns.barplot(x=plot_data_svc_neg.columns[3], y=plot_data_svc_neg.index, data=plot_data_svc_neg, ax = ax6,
            label="Total", color="r")

ax2.set(yticklabels=[])
#ax3.set(yticklabels=[])
ax4.set(yticklabels=[])
ax6.set(yticklabels=[])

ax3.set_xlim(0,60)
ax5.set_xlim(-60,0)
ax4.set_xlim(0,360)
ax6.set_xlim(-360,0)

ax1.set(xlabel='(a)', title='Gradient Boost')
ax2.set(xlabel='(b)', title='Random Forest')
ax3.set(xlabel='(c)', title='Linear SVC')
ax4.set(xlabel='(d)', title='Logistic Regression')
ax5.set(xlabel='(e)', title='Linear SVC')
ax6.set(xlabel='(f)', title='Logistic Regression')

plt.tight_layout(pad=1.0)

plt.rcParams['figure.figsize'] = [6, 4]